In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pathlib
import kaleido
import plotly.express as px
import plotly.io as pio
from IPython.display import display, Markdown

pd.set_option('future.no_silent_downcasting', True)

In [ ]:
# borne de recharge
data =  pd.read_csv(
    ".\data\consolidation-etalab-schema-irve-statique-v-2.3.1-20240705.csv",
    low_memory=False
    ,parse_dates=["date_mise_en_service"]
    ,dtype={"consolidated_code_postal": str
                       })\
  .sort_values(['id_pdc_itinerance', 'last_modified'])\

# Département
df_dep = pd.read_csv(
    ".\data\departements-france.csv"
    ,dtype={"code_departement": str
                        })

data.head()

In [ ]:
print(data.shape)
print(data.dtypes)

In [ ]:
# labéliser les colonnes pour les visuels
labels = {
    "nom_amenageur": "Nom amenageur"
    ,"nom_operateur": "Nom opérateur"
    ,"nom_enseigne": "Nom enseigne"
    ,"mis_en_service_cette_annee": "Mise en service en 2023"
    ,"date_mise_en_service": "Date de mise en service"
    ,"puissance_nominale_cat": "Puissance nominale"
    ,"prise_type_combo_ccs": "Possède une prise combo"
    ,"prise_type_2": "Possède une prise type 2"
}

In [ ]:
plt.figure(figsize=(40,20))
sns.heatmap(data.isna(), cbar=False)

In [ ]:
# Calculer les pourcentages de valeurs manquantes
missing_percentages = (data.isna().sum() / data.shape[0]).sort_values(ascending=False)

# Filtrer les colonnes avec un pourcentage de valeurs manquantes différent de zéro
filtered_missing_percentages = missing_percentages.loc[missing_percentages > 0]

# Afficher les pourcentages de valeurs manquantes filtrés
print(filtered_missing_percentages)

In [ ]:
# Suppression des colonnes qui ont plus que 50% des données manquantes
data = data.drop(columns=['id_pdc_local', 'observations', 'cable_t2_attache', 'tarification', 'num_pdl'])

# Suppression des colonnes qui ne peuvent pas etre remplises () et surtout ils ont une pourcentage importante manquante 
data =  data.drop(columns=['id_station_local'])

# le pourcentage des valeurs manquantes dans les colonnes suivantes est faible 
data.dropna(subset=['contact_amenageur', 'telephone_operateur'], inplace=True)


In [ ]:
# Verifier les pourcentages après les drop
# Shows the precentge of missing values in each column
(data.isna().sum()/data.shape[0]).sort_values(ascending=True)

In [ ]:
# Convertir les valeurs en chaînes de caractères, obtenir les valeurs uniques et les trier par ordre ascendant
unique_data_paiment_cb = sorted(data['paiement_cb'].astype(str).unique())
unique_data_paiment_autre = sorted(data['paiement_autre'].astype(str).unique())
unique_data_gratuit = sorted(data['gratuit'].astype(str).unique())
unique_data_raccordemant = sorted(data['raccordement'].astype(str).unique())

# Afficher les valeurs uniques
print(unique_data_paiment_cb)
print('\n')
print(unique_data_paiment_autre)
print('\n')
print(unique_data_gratuit)
print('\n')
print(unique_data_raccordemant)

In [ ]:
# fill it with random values from the column itself paiment_cb, paiment_autre, raccordemant 

In [ ]:
df_filter = data["id_pdc_itinerance"]=="Non concerné"

data = pd.concat([
  data[~df_filter].drop_duplicates('id_pdc_itinerance', keep='last'),
  data[df_filter]
])

In [ ]:
# Calcul approximatif du nombre de PDC par année
# Extraire l'année à partir de la colonne de dates
data['annee'] = data['date_mise_en_service'].dt.year

# Compter le nombre de points de recharge par année
count_by_year = data['annee'].value_counts().sort_index()

# Afficher le résultat
print("Nombre de points de recharge par année :")
print(count_by_year)

In [ ]:
# Date de mise en service
# Remplacer les valeurs de mise en service avant 2010 par None
data.loc[data["date_mise_en_service"]<pd.to_datetime("2010-01-01"),"date_mise_en_service"]=None
# Vérifier si la mise en service a eu lieu en 2024 en ajoutant une colonne 'mis_en_service_cette_annee'
data["mis_en_service_cette_annee"] = np.where(data["date_mise_en_service"].dt.year==pd.to_datetime("2024-01-01").year
         , "Oui"
         , "Non")

In [ ]:
# Power
# Normaliser les valeurs de la puissance et les catégoriser
data["puissance_nominale_cat"] = pd.cut(\
  data["puissance_nominale"].apply(lambda x: x/1000 if x >1000 else x)\
,[0,1.8,3.5,7.5,26,52,151,500]
,labels=["1.7","3.4","7.5","22","50","150",">150"]
, include_lowest=False)

In [ ]:
# les types de prise 
data["prise_type_combo_ccs"] = (data["prise_type_combo_ccs"].str.lower().map
                      ({"0":"Non","false":"Non","1":"Oui","true":"Oui"}))
data["prise_type_2"] = (data["prise_type_2"].str.lower().map
                      ({"0":"Non","false":"Non","1":"Oui","true":"Oui"}))
# les types de paiment 
data["paiement_autre"] = (data["paiement_autre"].str.lower().map
                      ({"0":"FALSE","false":"FALSE","False":"FALSE","1":"TRUE","true":"TRUE","True":"TRUE"}))
data["paiement_cb"] = (data["paiement_cb"].str.lower().map
                      ({"0":"FALSE","false":"FALSE","False":"FALSE","1":"TRUE","true":"TRUE","True":"TRUE"}))

data["gratuit"] = (data["gratuit"].str.lower().map
                      ({"0":"FALSE","false":"FALSE","False":"FALSE","1":"TRUE","true":"TRUE","True":"TRUE"}))



In [ ]:
# Ajout d'une colonne 'code_departement' à partir de 'consolidated_code_postal'
data["code_departement"] = data["consolidated_code_postal"].str[:2]

# Join des deux tables sur la colonne 'code_departement'
data = pd.merge(data,df_dep, how='left', on='code_departement')

In [ ]:
data.head(100)

In [ ]:
# Shows the precentge of missing values in each column
(data.isna().sum()/data.shape[0]).sort_values(ascending=True)

In [ ]:
# Convertir les valeurs en chaînes de caractères, obtenir les valeurs uniques et les trier par ordre ascendant
unique_data_paiment_cb = sorted(data['paiement_cb'].astype(str).unique())
unique_data_paiment_autre = sorted(data['paiement_autre'].astype(str).unique())
unique_data_gratuit = sorted(data['gratuit'].astype(str).unique())
unique_data_raccordemant = sorted(data['raccordement'].astype(str).unique())

# Afficher les valeurs uniques
print(unique_data_paiment_cb)
print('\n')
print(unique_data_paiment_autre)
print('\n')
print(unique_data_gratuit)
print('\n')
print(unique_data_raccordemant)

In [ ]:
# le but de la fonction c'est de remplacer les valeurs nan par des randoms values pour les remplir 
def replace_nan_with_random(df, column):
    # Sélectionner les valeurs non manquantes
    non_nan_values = df[column].dropna().unique()
    # Générer des valeurs aléatoires pour remplacer les NaN
    random_values = np.random.choice(non_nan_values, size=df[column].isna().sum(), replace=True)
    # Remplacer les NaN par les valeurs aléatoires générées
    df.loc[df[column].isna(), column] = random_values
    

In [ ]:
# Remplacer les valeurs NaN par des valeurs aléatoires pour chaque colonne spécifiée
for column in ['paiement_cb', 'paiement_autre', 'raccordement']:
    replace_nan_with_random(data, column)

In [ ]:
# Shows the precentge of missing values in each column
(data.isna().sum()/data.shape[0]).sort_values(ascending=True)